In [1]:
import os

In [2]:
os.getcwd()

'C:\\Users\\Lenovo\\ALL Python ML notebooks\\Deep Learning Notebooks'

In [5]:
os.chdir('C:/Users/Lenovo/ALL Python ML notebooks/Deep Learning Notebooks/Casting Data')

In [6]:
os.getcwd()

'C:\\Users\\Lenovo\\ALL Python ML notebooks\\Deep Learning Notebooks\\Casting Data'

### Casting Product Image Data For Quality Inspection

* Convolutional neural networks (CNN) is used to convert digital image content into a single vector of numbers(numeric vector) representing the unique characteristics of the image. 
* The column of numbers is input to a dense fully connected Neural Network layer against the labels, which image is cat, which image is bird etc.
* The classification model learns these numeric vector inputs against the labels

In [7]:
![image.png](attachment:image.png)

'[image.png]' is not recognized as an internal or external command,
operable program or batch file.


### Reading the Images data

In [8]:
# Deep Learning CNN model to recognise mechanical part which is defective/okay
'''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not''''''This script uses a database of images and creates CNN model on top of it to test
   if the given image is recognized correctly or not'''

'''########################## IMAGE PRE-PROCESSING for TRAINING and TESTING data ##############################'''

TrainingImagePath = 'C:/Users/Lenovo/ALL Python ML notebooks/Deep Learning Notebooks/Casting Data/casting_data/train'

In [13]:
import numpy as np
import pandas as pd
from keras.preprocessing.image import ImageDataGenerator
# To Understand more about ImageDataGenerator at below link
# https://blog.keras.io/building-powerful-image-classification-models-using-very-little-data.html

In [14]:
# Defining pre-processing transformation on raw images of training data
# This ImageDataGenerator tries to generate more data than existed
# train datagen will have these kind of the properties
train_datagen = ImageDataGenerator(
        rescale=1/255, # Rescale to 255 pixels
        shear_range=0.1, # Shear/cut it by 10%
        zoom_range=0.1, # Zoom it by 10%
        horizontal_flip=True) # Flip it horizontally as well,
# It applies above operations on the available images

In [15]:
# Defining pre-processing transformations on raw images of testing data
# In testing we are not performing any operation
test_datagen = ImageDataGenerator(rescale=1/255)

In [16]:
# Generating the Training Data generated by using TrainingDataGenerator object
training_set = train_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64, 64), #Usually we have 331x331 pixels image which are more enough for the system to process.
                            # Essentially these algorithms are helpful to preserve main characteristic of images, still
                            # generate a good image, here we compress it and learn these 331 pixels
                            # Now CNN has reinforced it, compress it and extracting and learning it..
                            # Just to avoid CPU load, we are reducing the image spec, which is good in recognising
                            # If we need, we can give 200x200 but it should be as lesser as possible
        batch_size=32, # batch size is 32, in how many batches we are going to do operation
        class_mode='categorical') # It maintains an index, face1->index 0, etc

Found 6633 images belonging to 2 classes.


In [18]:
# Generating the Testing Data
test_set = test_datagen.flow_from_directory(
        TrainingImagePath,
        target_size=(64,64),
        batch_size=32,
        class_mode='categorical')

Found 6633 images belonging to 2 classes.


In [19]:
# Printing class labels for each product photo
test_set.class_indices

{'def_front': 0, 'ok_front': 1}

### Creating a list of faces and classes

In [22]:
'''################ Creating lookup table for all photos###########'''
# class_indices have the numeric tag for each photo
TrainClasses=training_set.class_indices

# Storing the face and the numeric tag forr future reference
ResultMap={}
for partValue, partName in zip(TrainClasses.values(), TrainClasses.keys()):
    ResultMap[partValue]=partName
    
# Saving the face map for future reference
import pickle
with open("ResultMap.pkl", 'wb') as f: # here we storing it
    pickle.dump(ResultMap, f, pickle.HIGHEST_PROTOCOL)
    
print('Mapping of Part and its ID', ResultMap)

# The number of neurons for the output layer is equal to the number of parts
OutputNeurons=len(ResultMap)
print('\ The Number of output neurons: ', OutputNeurons) # Output layer is determined by number of classes, 2 probabilities
# No. of output neurons are 2, output produced by ANN is 2 probabilities out of which highest probabilities will win

Mapping of Part and its ID {0: 'def_front', 1: 'ok_front'}
\ The Number of output neurons:  2


### Creating the CNN model

In [23]:
'''############## Create CNN Deep Learning model ##########'''
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense

'''Initializing the Convolutional Neural Network'''
classifier= Sequential()

''' STEP--1 CONVOLUTION
# Adding the first layer of CNN
# We are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64x64) pixels representing Red, Green and Blue components of pixels'''
classifier.add(Convolution2D(filters=32, kernel_size=(3,3), strides=(1,1), input_shape=(64,64,3),# Means 3-> Matrices of 64x64
                            activation='relu'))

'''# STEP--2 MAX Pooling'''# Hyper parameter-> pool size which affects accuracy
classifier.add(MaxPool2D(pool_size=(2,2)))

'''############ ADDITIONAL LAYER of CONVOLUTION for better accuracy ##########'''
# Additional round of convule/consize/reduce image size-> it extracts most important part of the image
classifier.add(Convolution2D(filters=64, kernel_size=(5,5), strides=(2,2), activation='relu'))

classifier.add(MaxPool2D(pool_size=(2,2)))


'''# STEP--3 FLATTEING'''
classifier.add(Flatten())

'''# STEP--4 FULLY CONNECTED NEURAL NETWORK'''
classifier.add(Dense(OutputNeurons, activation='softmax'))

'''# COMPILING THE CNN'''
#classfier.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
# crossentropy--> categorical_crossentropy for multiclass classification
# binary_crossentropy--> for binary classification

### Fitting the model

In [24]:
import time
# Measuring the time taken by the model to train
StartTime=time.time()

# Starting the model training
classifier.fit_generator(
                    training_set,
                    steps_per_epoch=5,
                    epochs=35, # like batch size for you, change epoch size to optimize the accuracy
                    validation_data=test_set,
                    validation_steps=2)

EndTime=time.time()
print('############# Total Time Taken: ', round((EndTime-StartTime)/60), 'Minutes ########')

C:\Users\Lenovo\AppData\Local\Temp\ipykernel_7240\201778491.py:6: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  classifier.fit_generator(


Epoch 1/35


5/5 [==============================] - 7s 1s/step - loss: 0.7695 - accuracy: 0.4500 - val_loss: 0.7190 - val_accuracy: 0.4375
Epoch 2/35
5/5 [==============================] - 5s 1s/step - loss: 0.7040 - accuracy: 0.4875 - val_loss: 0.6762 - val_accuracy: 0.5625
Epoch 3/35
5/5 [==============================] - 4s 852ms/step - loss: 0.7063 - accuracy: 0.4812 - val_loss: 0.6908 - val_accuracy: 0.3906
Epoch 4/35
5/5 [==============================] - 4s 827ms/step - loss: 0.6830 - accuracy: 0.6438 - val_loss: 0.6808 - val_accuracy: 0.7031
Epoch 5/35
5/5 [==============================] - 4s 784ms/step - loss: 0.6849 - accuracy: 0.5437 - val_loss: 0.6776 - val_accuracy: 0.5312
Epoch 6/35
5/5 [==============================] - 4s 788ms/step - loss: 0.6687 - accuracy: 0.5938 - val_loss: 0.6406 - val_accuracy: 0.6562
Epoch 7/35
5/5 [==============================] - 4s 764ms/step - loss: 0.6830 - accuracy: 0.5375 - val_loss: 0.6723 - val_accuracy: 0.5938
Epoch 8/35
5/5 [========

### Saving the model

In [27]:
### Saving the model
classifier.save('C:/Users/Lenovo/ALL Python ML notebooks/Deep Learning Notebooks/Casting Data')

INFO:tensorflow:Assets written to: C:/Users/Lenovo/ALL Python ML notebooks/Deep Learning Notebooks/Casting Data\assets


INFO:tensorflow:Assets written to: C:/Users/Lenovo/ALL Python ML notebooks/Deep Learning Notebooks/Casting Data\assets


### Testing the model on a different part

In [37]:
'''############## Making single predictions #############'''
import numpy as np
from keras.preprocessing import image

testImage='C:/Users/Lenovo/ALL Python ML notebooks/Deep Learning Notebooks/Casting Data/casting_data/test/ok_front/cast_ok_0_504.jpeg'
test_image=image.load_img(testImage,target_size=(64,64)) # Converts image to array object like nuerical array
test_image=image.img_to_array(test_image) # the image 64,64 has to be converted into array format

test_image=np.expand_dims(test_image,axis=0) # 64x64x3 array is created
# This image is at the same level, where we trained our model

result=classifier.predict(test_image,verbose=0) # Passing test image to classifier using predict function
# verbose is 0 which doesn't produce the log output

print(result)
print(ResultMap)

print('####'*10)
print('Prediction is: ', ResultMap[np.argmax(result)]) # which is the max value give me the location of the value
# whichever is maximum, it gives the location of that value , from which we get index, result map of highest value will
# give us the part number

[[0. 1.]]
{0: 'def_front', 1: 'ok_front'}
########################################
Prediction is:  ok_front
